In [ ]:
import string
import re
import json

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import mapclassify

import matplotlib as mpl
import matplotlib.cm as cm

from graphly.api_client import SparqlClient

# TODO: 
# 1. Ask Geoimpact for list of "Firmen mit eigene Transformatorenstation"
# 2. Join query with swisstopo
# 3. New notebook: Electricity prices over time. Were market prices reflected on end consumers ? 

In [ ]:
ENDPOINT = "https://int.lindas.admin.ch/query"

sparql = SparqlClient(ENDPOINT)
sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "property": "<https://ld.stadt-zuerich.ch/statistics/property/>",
    "measure": "<https://ld.stadt-zuerich.ch/statistics/measure/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "ssz": "<https://ld.stadt-zuerich.ch/statistics/>"
})

In [ ]:
query = """
PREFIX cube: <https://cube.link/>
PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
PREFIX schema: <http://schema.org/>

SELECT ?municipality_id ?category ?energy ?grid ?aidfee (?community_fees + ?aidfee as ?taxes) ?fixcosts ?variablecosts 
FROM <https://lindas.admin.ch/elcom/electricityprice>
WHERE {
    <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.
    
    ?observation
      elcom:category/schema:name ?category;
      elcom:municipality ?municipality_id;
      elcom:period "2020"^^<http://www.w3.org/2001/XMLSchema#gYear>;
      elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
      elcom:fixcosts ?fixcosts;
      elcom:total ?variablecosts;
      elcom:gridusage ?grid;
      elcom:energy ?energy;
      elcom:charge ?community_fees;
      elcom:aidfee ?aidfee.
      
}
ORDER BY ?muncipality ?category ?variablecosts
"""

prices = sparql.send_query(query)
prices = prices.groupby(["municipality_id", "category"]).first().reset_index()
prices.head()

In [ ]:
query = """
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?category ?description
WHERE {
  GRAPH <https://lindas.admin.ch/elcom/electricityprice> {
    
    ?s <https://energy.ld.admin.ch/elcom/electricityprice/dimension/category> ?category_uri.
    ?category_uri schema:name ?category .
    ?category_uri schema:description ?description .
  }
}
ORDER BY ?category
"""
df = sparql.send_query(query)
df.head()

In [ ]:
def extract_consumption(description: str) -> int:
    """
    Extract average electricity consumption from a description.
    Args:
        description:  Category description for electricity prices
        
    Returns: 
        int:          Electricity consumption in kWh/year
    
    """
    
    number_as_string = description.split(" kWh/Jahr")[0]
    return int(number_as_string.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
cat2description = dict(zip(df.category, df.description))
cat2consumption = dict(zip(df.category, [extract_consumption(d) for d in df.description]))

prices["consumption"] = prices[["category"]].replace({"category": cat2consumption})
prices["monthly_bill"] = (prices.consumption*prices.variablecosts/12 + prices.fixcosts)/100
prices.head()

In [ ]:
geosparql = SparqlClient("https://ld.geo.admin.ch/query")

query = """
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX schema: <http://schema.org/>
PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    
SELECT ?municipality_id ?municipality ?population ?boundary 

WHERE {
  ?muni_iri dct:hasVersion ?version ;
            geonames:featureCode geonames:A.ADM3 .
  
  ?version schema:validUntil "2020-12-31"^^<http://www.w3.org/2001/XMLSchema#date>;
           geonames:population ?population;
           schema:name ?municipality;
           geosparql:hasGeometry/geosparql:asWKT ?boundary.
  
  BIND(IRI(REPLACE(STR(?muni_iri), "https://ld.geo.admin.ch/boundaries/", "https://ld.admin.ch/")) AS ?municipality_id)
}

"""
communes = geosparql.send_query(query)
communes = communes.set_crs(epsg=4326)
communes.head()

In [ ]:
communes.plot()

In [ ]:
join = pd.merge(communes[["municipality_id", "municipality", "population", "boundary"]], prices, how="inner", on="municipality_id")
join.drop(columns=["variablecosts", "consumption"], inplace=True)
join.head()

In [ ]:
def plot_prices_heatmap(category, color_palette, variable, legend_label, N):
    
    colors = sns.color_palette(color_palette, n_colors=N).as_hex()
    
    df = join[join["category"] == category]
    df = df.set_index("municipality")

    classifier = mapclassify.NaturalBreaks(y=df[variable], k=N)
    df["buckets"] = df[[variable]].apply(classifier)
    labels = mapclassify.classifiers._get_mpl_labels(classifier, fmt="{:.0f}")
    labels = ["-".join(re.findall(r"\d+", l)) for l in labels]
    bucket2label = dict(zip(range(N), labels))
    df = df.replace({"buckets": bucket2label})

    colormap={bucket2label[i]: color for i, color in enumerate(colors)}

    fig = px.choropleth(df, geojson=json.loads(df.to_json()), locations=df.index, 
                        color="buckets",
                        color_discrete_map=colormap,
                        projection="transverse mercator",
                        hover_name=df.index,
                        hover_data={"buckets": False, variable: ':.2f'},
                        title="Electricity Prices: {}".format(cat2description[category]),
                        labels={"buckets": legend_label})
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"l":0,"b":0})
    fig.update_traces(marker_line_width=0)
    fig.show()
    

In [ ]:
colors="YlGn"
plot_prices_heatmap("H2", colors, "monthly_bill", "average monthly bill [CHF]", 5)

In [ ]:
plot_prices_heatmap("C1", colors, 'monthly_bill', "average monthly bill [CHF]", 5)

In [ ]:
# TODO: same map for different tariffs (side-to-side)
# https://plotly.com/python/map-subplots-and-small-multiples/

### Competition between energy providers

These municipalities have more than one electricity provider

In [ ]:
query = """
PREFIX cube: <https://cube.link/>
PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
PREFIX schema: <http://schema.org/>

SELECT ?municipality_id (MAX(?providers) AS ?providers)
FROM <https://lindas.admin.ch/elcom/electricityprice>
WHERE {
  SELECT ?municipality_id ?category (COUNT(*) AS ?providers)

  WHERE {
      <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.

      ?observation
        elcom:category/schema:name ?category;
        elcom:municipality ?municipality_id;
        elcom:period "2020"^^<http://www.w3.org/2001/XMLSchema#gYear>;
        elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
        elcom:energy ?energy.
  }
  GROUP BY ?municipality_id ?category
  HAVING (COUNT(*) > 1)
}
GROUP BY ?municipality_id
"""
df = sparql.send_query(query)
df.head()

Let's get boundaries for these municipalities

In [ ]:
join2 = pd.merge(communes[["municipality_id", "municipality", "population", "boundary"]], df, how="outer", on="municipality_id").fillna(1)
join2.head()

In [ ]:
join2.providers = join2.providers.astype(int).astype(str)
join2 = join2.set_index("municipality")
data = json.loads(join2.to_json())

In [ ]:
def num2color(value, minimum, maximum):
    return mpl.colors.rgb2hex(cm.plasma((value-minimum)/maximum))
    
providers = join2.providers.unique()
colormap = {i: num2color(int(i), 1, len(providers)-1) for i in providers}

In [ ]:
fig = px.choropleth(join2, geojson=data, locations=join2.index, 
                    color="providers",
                    color_discrete_map=colormap,
                    projection="transverse mercator",
                    hover_name=join2.index,
                    #hover_data={"buckets": False},
                    labels={"providers": "Electricity providers"})
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.show()

### Grid costs and population density 
=> which region is the most expensive to maintain
Does that correlate with area?

In [ ]:
join["hectares"] = join.to_crs(epsg=3035).area/10000 # In hectares
join["population_density"] = join["population"]/join["hectares"]
join.head(3)

In [ ]:
# TO DO: histogram
# x - price
# y - frequency
var = "monthly_bill"
dff = join[join.category == "H1"][["population", var]]
dff[var] = dff[var].round(2)

dff = dff.groupby([var]).sum().reset_index()
dff["frequency"] = dff["population"]/dff["population"].sum()*100

dff.head()

In [ ]:
fig = px.histogram(dff, x=var, y="frequency", nbins=10)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
plot_prices_heatmap("C1", "YlOrRd", 'population_density', "Inhabitants per ha", 6)

In [ ]:
plot_prices_heatmap("C1", "YlOrRd", 'grid', "Grid usage [Rp/kWh]", 6)

In [ ]:
# Relationship between grid costs, and population density
dff = join[join.category == "C1"]

fig = px.scatter(dff, y="grid", x="population_density", hover_data=["municipality"],
                labels={
                     "population_density": "Inbahitants per ha",
                     "grid": "Grid costs per kWh"})
fig.show()

In [ ]:
join = join.assign(pays_grid=lambda x: x.grid!=0, 
                   pays_aidfee=lambda x: x.aidfee!=0,
                   pays_energy=lambda x: x.energy!=0)

join.head(3)

In [ ]:
df = join[join.category=="C1"]
df = df.set_index("municipality")
data = json.loads(df.to_json())

In [ ]:
fig = px.choropleth(df, geojson=data, locations=df.index, 
                    color="pays_grid",
                    projection="transverse mercator",
                    hover_name=df.index,
                    #hover_data={"buckets": False},
                    labels={"pays_for_grid": "Paid grid usage"})
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
fig = px.choropleth(df, geojson=data, locations=df.index, 
                    color="pays_aidfee",
                    projection="transverse mercator",
                    hover_name=df.index,
                    labels={"pays_aidfee": "Pays aidfee"},
                    color_discrete_map={True: "#00CC96", False: "#AB63FA"}
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
fig = px.choropleth(df, geojson=data, locations=df.index, 
                    color="pays_energy",
                    projection="transverse mercator",
                    hover_name=df.index,
                    labels={"pays_energy": "Pays energy"}
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"l":0,"b":0}, title=cat2description["C4"])
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
join.energy.min()

In [ ]:
# These communes do not pay for grid

temp = join[["municipality_id", "municipality", "pays_grid", "population"]].groupby(["municipality_id", "municipality", "population"]).all().reset_index()
temp[(~temp.pays_grid)]

How many people profit from it?

In [ ]:
free_grid_pop = temp[(~temp.pays_grid)]["population"].sum()
total_pop = communes.population.sum()

px.pie(names=["Population with free grid access", "Population with paid grid access"], 
       values=[free_grid_pop,total_pop-free_grid_pop])

In [ ]:
# These communes do not pay aidfee
temp = join[["municipality_id", "municipality", "pays_aidfee", "population"]].groupby(["municipality_id", "municipality", "population"]).all().reset_index()
temp[(~temp.pays_aidfee)]

In [ ]:
free_aidfee_pop = temp[(~temp.pays_aidfee)]["population"].sum()

px.pie(names=["Population not paying aidfee", "Population paying aidfee"], 
       values=[free_aidfee_pop,total_pop-free_aidfee_pop])

In [ ]:
# These communes do not pay for energy
join["has_free_energy"] = ~join.pays_energy
temp = join[["municipality_id", "municipality", "has_free_energy", "population"]].groupby(["municipality_id", "municipality", "population"]).all().reset_index()
temp[temp.has_free_energy]

Ok. There is no comune where all tariffs have free energy. But are there any places with free electricty?

In [ ]:
join[["municipality_id", "municipality", "category", "pays_energy"]][~(join.pays_energy)].reset_index(drop=True)

In [ ]:
for cat in join["category"][~(join.pays_energy)].unique():
    print("- ", cat2description[cat])

Are there any companies that profit from it?

Zefix: find companies registered in this region (and hence entitiled to free elencticity)
Use only AGs

Switzerland has around 2500 enterprises with its won transformators => they could benefit from these locations

In [ ]:
query = """
SELECT ?company ?type ?company_iri
WHERE {
    ?company_iri a <https://schema.ld.admin.ch/ZefixOrganisation>.
    ?company_iri <http://schema.org/legalName> ?company.
    ?company_iri <https://schema.ld.admin.ch/municipality> <https://ld.admin.ch/municipality/3695>.
    ?company_iri <http://schema.org/additionalType>/<http://schema.org/name> ?type .
  
  FILTER(LANG(?type) = "de")
}
ORDER BY ?type ?company
"""
df = sparql.send_query(query)
df

In [ ]:
municipalities_free_energy = join["municipality_id"][~(join.pays_energy)].unique()

query = """
SELECT ?municipality ?company ?company_iri 
WHERE {{
	?company_iri a <https://schema.ld.admin.ch/ZefixOrganisation>.
  	?company_iri <http://schema.org/legalName> ?company.
    ?company_iri <https://schema.ld.admin.ch/municipality> ?municipality_iri.
    ?company_iri <http://schema.org/additionalType> <https://ld.admin.ch/ech/97/legalforms/0106> .
  
  ?municipality_iri <http://schema.org/name> ?municipality.
  
  FILTER(?municipality_iri IN({}))
  
}}
ORDER BY ?municipality ?company
""".format("<" + ">,<".join(municipalities_free_energy) + ">")
df = sparql.send_query(query)
df

## Join DFs

In [ ]:
query = """
PREFIX cube: <https://cube.link/>
PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
PREFIX schema: <http://schema.org/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX geonames: <http://www.geonames.org/ontology#>
PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>

SELECT ?municipality ?category ?energy ?grid (?community_fees + ?aidfee as ?taxes) ?fixcosts ?variablecosts 
FROM <https://lindas.admin.ch/elcom/electricityprice>
WHERE {
    <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.
    
    ?observation
      elcom:category <https://energy.ld.admin.ch/elcom/electricityprice/category/H1>;
      elcom:municipality ?muni_iri;
      elcom:period "2020"^^<http://www.w3.org/2001/XMLSchema#gYear>;
      elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
      elcom:fixcosts ?fixcosts;
      elcom:total ?variablecosts;
      elcom:gridusage ?grid;
      elcom:energy ?energy;
      elcom:charge ?community_fees;
      elcom:aidfee ?aidfee.
    
    BIND(IRI(REPLACE(STR(?muni_iri),"https://ld.admin.ch/", "https://ld.geo.admin.ch/boundaries/")) AS ?municipality) .
  
    {SERVICE <https://ld.geo.admin.ch/query> {
      SELECT ?municipality ?name ?population ?polygon WHERE {
        
        ?municipality dct:hasVersion ?version ;
                      geonames:featureCode geonames:A.ADM3 .
        
        ?version schema:validUntil "2020-12-31"^^<http://www.w3.org/2001/XMLSchema#date>;
             geonames:population ?population ;
             schema:name ?name .
        
        ?version geosparql:hasGeometry/geosparql:asWKT ?polygon
    }
    }}
}
LIMIT 500
"""

#df = sparql.send_query(query)
#df.head()